In [126]:
import numpy as np
import pandas as pd

In [127]:
df = pd.read_csv('un.csv.1')

In [128]:
df.shape

(207, 14)

In [129]:
df.head()

,country,region,tfr,contraception,educationMale,educationFemale,lifeMale,lifeFemale,infantMortality,GDPperCapita,economicActivityMale,economicActivityFemale,illiteracyMale,illiteracyFemale
0,Afghanistan,Asia,6.90,NaN,NaN,NaN,45.0,46.0,154,2848,87.5,7.2,52.800,85.00
1,Albania,Europe,2.60,NaN,NaN,NaN,68.0,74.0,32,863,NaN,NaN,NaN,NaN
2,Algeria,Africa,3.81,52,11.1,9.9,67.5,70.3,44,1531,76.4,7.8,26.100,51.00
3,American.Samoa,Asia,NaN,NaN,NaN,NaN,68.0,73.0,11,NaN,58.8,42.4,0.264,0.36
4,Andorra,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
df.count()

country                   207
region                    207
tfr                       197
contraception             144
educationMale              76
educationFemale            76
lifeMale                  196
lifeFemale                196
infantMortality           201
GDPperCapita              197
economicActivityMale      165
economicActivityFemale    165
illiteracyMale            160
illiteracyFemale          160
dtype: int64

In [131]:
type(df.count())

pandas.core.series.Series

In [132]:
df.count().sort_values(ascending=False)

region                    207
country                   207
infantMortality           201
GDPperCapita              197
tfr                       197
lifeFemale                196
lifeMale                  196
economicActivityFemale    165
economicActivityMale      165
illiteracyFemale          160
illiteracyMale            160
contraception             144
educationFemale            76
educationMale              76
dtype: int64

In [133]:
df.dtypes

country                    object
region                     object
tfr                       float64
contraception             float64
educationMale             float64
educationFemale           float64
lifeMale                  float64
lifeFemale                float64
infantMortality           float64
GDPperCapita              float64
economicActivityMale      float64
economicActivityFemale    float64
illiteracyMale            float64
illiteracyFemale          float64
dtype: object

In [134]:
from scipy.cluster.vq import vq, kmeans, whiten

clusters = []
k_range  = 10
whitened = whiten(df[['lifeMale', 'lifeFemale', 'infantMortality', 'GDPperCapita']].dropna())

for i in range(k_range):
    codebook, distortion = kmeans(whitened, i+1)
    clusters.append({ 'size': i+1, 'codebook': codebook, 'distortion': distortion })

In [135]:
clusters[2]

{'codebook': array([[ 5.13921828,  4.8017149 ,  2.45945885,  0.09605102],
        [ 6.87791691,  6.59946058,  0.72203844,  0.32270088],
        [ 7.55558256,  7.21575251,  0.19885574,  2.50972306]]),
 'distortion': 0.74630471929794706,
 'size': 3}

In [136]:
# Calculate the distance between each point and the centroids of each cluster.
features = df[['lifeMale', 'lifeFemale', 'infantMortality', 'GDPperCapita']].dropna()
features.shape

(188, 4)

In [137]:
len(clusters)

10

In [138]:
import math

def distance_from_centroid(centroid):
    return lambda pt: math.sqrt(math.pow(centroid[0] - pt[0], 2) +
                                math.pow(centroid[1] - pt[1], 2) +
                                math.pow(centroid[2] - pt[2], 2) +
                                math.pow(centroid[3] - pt[3], 2))

# Probably want to use the whitened data to get meaningful results.
# Distances will be a M by k data frame.
for i in range(k_range):
    clusters[i]['distances'] = pd.DataFrame(index=features.index)
    for j in range(clusters[i]['size']):
        clusters[i]['distances']['d{}'.format(j+1)] = map(distance_from_centroid(clusters[i]['codebook'][j]), whitened)

In [139]:
clusters[2]['distances'].head()

,d1,d2,d3
0,1.743161,4.659206,6.092094
1,3.082862,0.273071,2.619362
2,2.694156,0.500676,2.741168
5,1.028329,4.064415,5.667586
7,3.564708,0.732580,1.727036


In [140]:
def get_closest(distances):
    return distances.idxmin()

def d_squared(distances):
    return math.pow(distances.min(), 2)
    
for i in range(k_range):
    try:
        clusters[i]['distances'].drop(labels=['closest'], inplace=True, axis=1)
    except ValueError:
        pass
    clusters[i]['distances']['closest'] = clusters[i]['distances'].apply(func=get_closest, axis=1)
    clusters[i]['distances']['d_squared'] = clusters[i]['distances'].apply(func=d_squared, axis=1)

In [141]:
clusters[2]['distances'].head()

,d1,d2,d3,closest,d_squared
0,1.743161,4.659206,6.092094,d1,3.038609
1,3.082862,0.273071,2.619362,d2,0.074568
2,2.694156,0.500676,2.741168,d2,0.250677
5,1.028329,4.064415,5.667586,d1,1.057461
7,3.564708,0.732580,1.727036,d2,0.536673


In [142]:
clusters[2]['distances'].pivot(columns='closest', values='d_squared').head()

closest,d1,d2,d3
0,3.038609,NaN,NaN
1,NaN,0.074568,NaN
2,NaN,0.250677,NaN
5,1.057461,NaN,NaN
7,NaN,0.536673,NaN


In [143]:
clusters[2]['distances'].pivot(columns='closest', values='d_squared').mean()

closest
d1    1.098055
d2    0.489967
d3    0.898197
dtype: float64

In [144]:
# Calculate average within-cluster sum of squares for each centroid.
distortions = []
for i in range(k_range):
    ds = clusters[i]['distances']

TypeError: append() takes exactly one argument (0 given)